In [1]:
from keras.models import Model 
from keras.layers import Input,LSTM
import numpy as np

Using TensorFlow backend.


#### 参考:
 - [（原文）Understand the Difference Between Return Sequences and Return States for LSTMs in Keras](https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/)
 - [（译文）理解LSTM在keras API中参数return_sequences和return_state](https://blog.csdn.net/u011327333/article/details/78501054)

---
### 1.return_sequences = False && return_state = False 
```python 
lstm = LSTM(1)
```
Keras API 中，return_sequences和return_state默认就是false。此时只会返回一个hidden state 值。如果input 数据包含多个时间步，则这个hidden state 是最后一个时间步的结果

In [2]:
# define model
inputs = Input(shape = (3,1))
lstm = LSTM(1)(inputs)   # return_sequences = True & return_state = False
model = Model(inputs = inputs,outputs = lstm)

# define data and predict
data = np.array([0.1,0.2,0.3]).reshape([1,3,1])
output = model.predict(data)
output

array([[-0.13695435]], dtype=float32)

---
### 2.return_sequences=True && return_state=False 
``` python
LSTM(1,return_sequences = True)```   
输出所有时间步的hidden state。

In [3]:
# define model
inputs = Input(shape = (3,1))
lstm = LSTM(1,return_sequences = True)(inputs)   # return_sequences = True & return_state = False
model = Model(inputs = inputs,outputs = lstm)

# define data and predict
data = np.array([0.1,0.2,0.3]).reshape([1,3,1])
state_hs = model.predict(data)
state_hs

array([[[-0.01409399],
        [-0.03686725],
        [-0.06507621]]], dtype=float32)

---
### 3.return_sequences=False && return_state=True
```python 
lstm, state_h, state_c = LSTM(1,return_state = True)
```
lstm 和 state_h 结果都是 hidden state。在这种参数设定下，它们俩的值相同，都是最后一个时间步的 hidden state。 state_c 是最后一个时间步 cell state结果。

In [4]:
# define model
inputs = Input(shape = (3,1))
lstm, state_h, state_c = LSTM(1,return_state = True)(inputs)   # return_sequences = False & return_state = Ture
model = Model(inputs = inputs,outputs = [lstm, state_h, state_c])

# define data and predict
data = np.array([0.1,0.2,0.3]).reshape([1,3,1])
outputs = model.predict(data)
outputs

[array([[-0.01314915]], dtype=float32),
 array([[-0.01314915]], dtype=float32),
 array([[-0.02674036]], dtype=float32)]

---
### 4.return_sequences=True && return_state=True
```python 
lstm, state_h, state_c = LSTM(1,return_sequences = True, return_state = True)
```
此时，既输出全部时间步的 hidden state ，又输出 cell state：
- lstm 存放的就是全部时间步的 hidden state。
- state_h 存放的是最后一个时间步的 hidden state
- state_c 存放的是最后一个时间步的 cell state

In [5]:
# define model
inputs = Input(shape = (3,1))
lstm, state_h, state_c = LSTM(1,return_sequences = True, return_state = True)(inputs)   # return_sequences = True & return_state = True
model = Model(inputs = inputs,outputs = [lstm, state_h, state_c])

# define data and predict
data = np.array([0.1,0.2,0.3]).reshape([1,3,1])
outputs = model.predict(data)
outputs

[array([[[0.02849635],
         [0.08156213],
         [0.15450114]]], dtype=float32),
 array([[0.15450114]], dtype=float32),
 array([[0.2794191]], dtype=float32)]

可以看到state_h 的值和lstm的最后一个时间步的值相同。

---
## 总结   
```return sequences``` ：返回每个时间步的hidden state    
`return state` ：返回最后一个时间步的hidden state 和cell state   
`return sequences` 和 `return state` ：可同时使用，三者都输出   